# DS Practicum 1

## Question 1

**Problem:**
The data science problem we were given concerns airline performance from 1987 to present, as provided by the Bureau of Transportation Statistics. The dataset contains airlines and their correlated figures, including arrival and departure data, flight distances, delays, among others. We specifically focused on the subset of this data containing statistics from flights which departed from CA, AZ, and NV in 2019. Given this data, we were tasked with analyzing the air traffic at certain airports/regions and analyzing the air traffic at certain airports/regions, by processing and visualizing data to find key takeaways regarding patterns, performance, and summary statistics.

## Question 2

**Loading Data**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV with the correct filename
df = pd.read_csv("2019_ONTIME_REPORTING_FSW-1.csv")

# View the first 5 rows
df.head()

**Inspecting Data and Converting to Suitable Types**

In [ ]:
# inspect the dimensions
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# inspect data types
df.info()

**Dimensions** <br></br>There are 1897503 rows and 14 columns of data.  

**Changes in Data Types**
<br></br>
CARRIER_CODE, TAIL_NUM, FL_NUM, ORIGIN, ORIGIN_ST, DEST, and DEST_ST will be changed to category variables because they represent labels.  FL_DATE will be changed to datetime because it represents a date.  DEP_TIME, DEP_DELAY, ARR_TIME, ARR_DELAY, and ELAPSED_TIME will be changed to integers as well.

In [ ]:
# change FL_DATE to datetime
df["FL_DATE"] = pd.to_datetime(df["FL_DATE"], format="%m/%d/%y")

# change CARRIER_CODE, TAIL_NUM, FL_NUM, ORIGIN, ORIGIN_ST, DEST, DEST_ST to
# categorical variables
df["FL_NUM"] = df["FL_NUM"].astype("category")
df["CARRIER_CODE"] = df["CARRIER_CODE"].astype("category")
df["TAIL_NUM"] = df["TAIL_NUM"].astype("category")
df["ORIGIN"] = df["ORIGIN"].astype("category")
df["ORIGIN_ST"] = df["ORIGIN_ST"].astype("category")
df["DEST"] = df["DEST"].astype("category")
df["DEST_ST"] = df["DEST_ST"].astype("category")

# change DEP_TIME, DEP_DELAY, ARR_TIME, ARR_DELAY, ELAPSED_TIME to Int64
df["DEP_TIME"] = df["DEP_TIME"].astype("Int64")
df["DEP_DELAY"] = df["DEP_DELAY"].astype("Int64")
df["ARR_TIME"] = df["ARR_TIME"].astype("Int64")
df["ARR_DELAY"] = df["ARR_DELAY"].astype("Int64")
df["ELAPSED_TIME"] = df["ELAPSED_TIME"].astype("Int64")

# verify changes 
df.dtypes

In [ ]:
# verify changes by checking first 5 rows
df.head()
original_df = df.copy()

In [ ]:
# percentage of missing values
df.isnull().mean()*100

In [ ]:
# distribution of missing values in rows
df.isnull().sum(axis=1).sort_values(ascending=False).value_counts()

**Missing Values**<br></br>Based on this information, missing data is not widespread since only a small percentage of rows have missing data.  Out of the rows with missing data, most have 5 or 6 missing columns.  Since the columns with missing values are related to arrival, departure, and delay times and this is important to the data science question on air traffic in part 1, these rows should be dropped.

In [ ]:
# remove rows with null values
df = df.dropna()

# verify changes (only rows with 0 null values retained)
df.isnull().sum(axis=1).sort_values(ascending=False).value_counts()

**Invalid Data**

Invalid times of 2400 for DEP_TIME were corrected by changing them to 0 and updating the FL_DATE by 1 day.  Rows with times of 2400 were not excluded since the data is not necessarily incorrect, just inconsistent since 2400 means 24 hours later or the next day.  This means the flight actually departed at midnight the next day.

In [ ]:
# determine number of rows with invalid DEP_TIME
invalid_dep_time_df = df[df["DEP_TIME"] == 2400]
print(f"{invalid_dep_time_df.shape[0]} rows with invalid DEP_TIME values")
# display rows with invalid departure times to verify later
invalid_dep_time_df.head()

There are 241 rows with invalid departure times.

In [ ]:
# update rows with invalid DEP_TIME
dep_time_filter = df["DEP_TIME"] == 2400
df.loc[dep_time_filter, ["DEP_TIME", "FL_DATE"]] = (
    df.loc[dep_time_filter].assign(DEP_TIME=0, FL_DATE=lambda x: x["FL_DATE"] + pd.DateOffset(days=1))
)

# verify changes by checking that there are no rows with 2400 as DEP_TIME
new_dep_time_df = df[df["DEP_TIME"] == 2400]
print(f"{new_dep_time_df.shape[0]} rows with invalid DEP_TIME values")

# verify changes by checking that rows in dep_time_filter now have corrected dates
verify_dep_time_cleaning = invalid_dep_time_df.merge(df, on=["FL_NUM", "TAIL_NUM", "ARR_TIME", "DEP_DELAY"], how="inner", suffixes=("_before", "_cleaned"))
verify_dep_time_cleaning = verify_dep_time_cleaning[["FL_DATE_before", "DEP_TIME_before", "FL_DATE_cleaned", "DEP_TIME_cleaned"]]
verify_date_change = (verify_dep_time_cleaning["FL_DATE_cleaned"] - verify_dep_time_cleaning["FL_DATE_before"]).dt.days
verify_dep_change = verify_dep_time_cleaning["DEP_TIME_cleaned"] == 0
print("All dates with 2400 departure time corrected by 1 day: ", (verify_date_change == 1).all())
print("All dates with 2400 departure time corrected to 0: ", (verify_dep_change).all())

There are now 0 rows with invalid departure times.

The other columns were checked for potential invalid data but no such values were found.  Checked for invalid CARRIER_CODE (not exactly two characters), FL_NUM(not exactly 1 to 4 digits), ORIGIN/DEST (not exactly 3 uppercase letters), ORIGIN_ST/DEST_ST (not exactly 2 uppercase letters), and DISTANCE (negative distances).  

**Additional Variable Checks**

In [ ]:
# check for invalid CARRIER_CODE (not two characters)
carrier_code_check = df["CARRIER_CODE"].str.len() != 2
carrier_code_check_df = df[carrier_code_check]
print(f"There are {carrier_code_check_df.shape[0]} CARRIER_CODE entries that are not 2 characters")

There are no CARRIER_CODE entries with IDs that are not 2 characters.

In [ ]:
# check for invalid FL_NUM (not 1 to 4 digits)
fl_num_check = (df["FL_NUM"].astype("string").str.len() > 4) |  (df["FL_NUM"].astype("string").str.len() == 0)
fl_num_check_df = df[fl_num_check]
print(f"There are {fl_num_check_df.shape[0]} FL_NUM entries that are not 1 to 4 digits")

There are no FL_NUM entries with values that are not 1 to 4 characters.

In [ ]:
# check for invalid ORIGIN/DEST that does not have 3 characters
origin_check = df["ORIGIN"].astype("string").str.len() != 3
origin_check_df = df[origin_check]
print(f"There are {origin_check_df.shape[0]} ORIGIN entries that are not 3 characters.")
dest_check = df["DEST"].astype("string").str.len() != 3
dest_check_df = df[dest_check]
print(f"There are {dest_check_df.shape[0]} DEST entries that are not 3 characters.")

There are no ORIGIN or DEST entries with values that are not 3 characters long.

In [ ]:
# check for invalid ORIGIN_ST/DEST_ST that does not have 2 characters
originst_check = df["ORIGIN_ST"].astype("string").str.len() != 2
originst_check_df = df[originst_check]
print(f"There are {originst_check_df.shape[0]} ORIGIN_ST entries that are not 2 characters.")
destst_check = df["DEST_ST"].astype("string").str.len() != 2
destst_check_df = df[destst_check]
print(f"There are {destst_check_df.shape[0]} DEST_ST entries that are not 2 characters.")

There are no ORIGIN_ST or DEST_ST entries with values that are not 3 characters long.

In [ ]:
# check for invalid DISTANCE that is negative
dist_check = df["DISTANCE"] < 0
dist_check_df = df[dist_check]
print(f"There are {dist_check_df.shape[0]} DISTANCE entries with negative values")

There are no DISTANCE entries that have negative values

**High Level Overview of Dataset: Airline Carriers and Delays**

In [ ]:
print(f"There are {df['CARRIER_CODE'].unique().size} different airline carriers")

In [ ]:
# create new data frame with just departure delays and origin airports
origin_airport_delay_avg_df = df.groupby("ORIGIN_ST", observed=True)[["DEP_DELAY"]].mean()
# create new data frame with just arrival delays and destination airports
dest_airport_delay_avg_df = df.groupby("DEST_ST", observed=True)[["ARR_DELAY"]].mean()

In [ ]:
# create new data frame with just carrier codes and delays
carrier_delay_df_avg = df.groupby("CARRIER_CODE", observed=True)[["ARR_DELAY", "DEP_DELAY"]].mean()
carrier_delay_df_std = df.groupby("CARRIER_CODE", observed=True)[["ARR_DELAY", "DEP_DELAY"]].std()
carrier_delay_df_avg.head()


In [ ]:
# visualize results
carrier_delay_df_avg.plot(y=["ARR_DELAY", "DEP_DELAY"], kind="bar")
plt.title("Average Arrival and Departure Delay By Carrier")
plt.ylabel("Time (minutes)")
plt.xlabel("Carrier")
plt.xticks(rotation=45)
plt.show()

In [ ]:
carrier_delay_df_std.plot(y=["ARR_DELAY", "DEP_DELAY"], kind="bar")
plt.title("Standard Deviation Arrival and Departure Delay By Carrier")
plt.ylabel("Time (minutes)")
plt.xlabel("Carrier")
plt.xticks(rotation=45)
plt.show()

**Arrival and Departure Delay Analysis**
<br></br>
Based on the average delay data, carriers F9, YX, and B6 are the most prone to arrival along with departure delays.  The carriers with the least delays are DL, WN, and MO.  Examining the standard devation of delays across carriers shows that the carrier with the greatest variation in arrival and departure delays is YX of almost 70 minutes.  The carrier with the least variability in terms of delays is EV with just under 20 minutes.  Therefore, VW flights tend to have lower delays and have more consistent delay patterns.  In contrast, YX flights tend to have greater delays and less consistent delay patterns.  Generally across airline carriers, the variability of delay patterns are similar for arrival and departure delays.  This may imply that flight carriers have similar patterns of arrival and departure delays.  


In [ ]:
# visualize results
origin_airport_delay_avg_df.plot(y=["DEP_DELAY"], kind="bar")
plt.title("Average Departure Delay By Region")
plt.ylabel("Time (minutes)")
plt.xlabel("Origin Region")
plt.show()

In [ ]:
# visualize results
dest_airport_delay_avg_df.plot(y=["ARR_DELAY"], kind="bar")
plt.title("Average Arrival Delay By Region")
plt.ylabel("Time (minutes)")
plt.ylim(0, max(origin_airport_delay_avg_df['DEP_DELAY']))
plt.xlabel("Destination Region")
plt.show()

**Arrival and Departure Delay Analysis Continued**
<br></br>
Based on the average arrival and departure delays by region, WY demonstrated longer delays compared to other regions.  AK and SC had the shortest departure delays while CT and AK had the shortest arrival delays.  This data can be further analyzed to determine if certain routes or regions may be associated with longer carrier delays.  Significant airport delays may also contribute to increased airport traffic.  However, for the most part, regions have similar arrival and departure delays between 5 and 20 minutes.

**High Level Overview: Air Traffic at Certain Regions**

In [ ]:
air_traffic_overview_departing_df = df.groupby("ORIGIN_ST", observed=True).size()
air_traffic_overview_departing_df.plot(y=["ORIGIN_ST"], kind="bar")
plt.title("Departing Flights Across Regions in Past Year")
plt.ylabel("Number of Flights")
plt.xlabel("State")
plt.show()

In [ ]:
air_traffic_overview_arrival_df = df.groupby("DEST_ST", observed=True).size()
air_traffic_overview_arrival_df.plot(y=["DEST_ST"], kind="bar")
plt.title("Arrival Flights Across Regions in Past Year")
plt.ylabel("Number of Flights")
plt.xlabel("State")
plt.show()

**Air Traffic**
<br></br>
Based on the number of arriving and departing flights across regions in the past year worth of data, CA dominates in terms of total flights, followed by AZ and NV.  Therefore, air traffic is greatest at CA, AZ, and NV based on measures of total flights.  Several regions including AK, AL, and SC have significantly less arriving and departing flights in comparison.  Therefore, these states likely have the least amount of air traffic.

**Summary Statistics**

In [ ]:
avg_fl_date = df['FL_DATE'].mean()
mode_carrier_code = df['CARRIER_CODE'].mode().iloc[0]
mode_origin_st = df['ORIGIN_ST'].mode().iloc[0]
mode_dest_st = df['DEST_ST'].mode().iloc[0]
avg_dep_time = df['DEP_TIME'].mean()
avg_dep_delay = df['DEP_DELAY'].mean()
avg_arr_time = df['ARR_TIME'].mean()
avg_arr_delay = df['ARR_DELAY'].mean()
avg_distance = df['DISTANCE'].mean()
std_distance = df['DISTANCE'].std()
avg_elapsed_time = df['ELAPSED_TIME'].mean()
std_elapsed_time = df['ELAPSED_TIME'].std()

print(f"mean flight date: {avg_fl_date}")
print(f"mode carrier code: {mode_carrier_code}")
print(f"mode origin: {mode_origin_st}")
print(f"mode destination: {mode_dest_st}")
print(f"average departure time: {avg_dep_time}")
print(f"average departure delay: {avg_dep_delay}")
print(f"average arrival time: {avg_arr_time}")
print(f"average arrival delay: {avg_arr_delay}")
print(f"average distance: {avg_distance}")
print(f"std distance: {std_distance}")
print(f"average elapsed time: {avg_elapsed_time}")
print(f"std elapsed time: {std_elapsed_time}")

**General Statistics**
<br></br>
Based on the summary statistics, the average flight date was 7/3/2019, indicating that the flight data has a relatively even spread since July is close to the middle of the year.  The most common carrier code was WN, which based on the previous data on delays, seems to have some of the lowest arrival and departure delays compared to other carriers.  Therefore, WN seems to be the least prone to delays.  The mode origin and destination were both CA, indicating that there is likely high air traffic in this region due to the number of incoming as well as outgoing flights.  Both arrival and departure times tend to have a similar average delay, which was also reflected in the previous charts.  The average distance is 1146.86 and this can vary by about 756.42 based on the standard deviation.  This can be reviewed on a region basis since longer average distances might indicate the presense of more international flights at an airport.  Lower average distances might indicate a more regional airport with high or low traffic based on other measures like the number of arriving or departing flights. The average elapsed time is 174.81 and the standard deviation of this time is 92.73.  Similar to the distances, this can be reviewed on a region basis to further determine air traffic at specific locations.

## Question 3

1. Explore patterns in the region i.e. the 3 states (AZ, NV, CA). Determine which region has the most air traffic. Comment on your findings and visualize the results. Note: it's important to explain your rational for determining what is "the most air traffic". For example, are you evaluating the frequency of flights or total flight time. Justify your response.

In [ ]:
# filter data for AZ, NV, and CA
states = ['AZ', 'NV', 'CA']
state_filter = df['ORIGIN_ST'].isin(states) | df['DEST_ST'].isin(states)
filtered_df = df[state_filter]
# verify results by checking how many rows don't have an original location or destination of one of the specified states
invalidRows = filtered_df[~df['ORIGIN_ST'].isin(states) & ~df['DEST_ST'].isin(states)].shape[0]
print(f"There are {invalidRows} rows without an original location or destination of AZ, NV, or CA")

In [ ]:
# determine time span of data
filtered_df['FL_DATE'].value_counts(ascending=True)

**Criteria For Most Air Traffic**
<br></br>
The region with the most air traffice will be determined by analyzing the frequency of flights that have arrived at that region and the frequency of flights that have departed from that region between August 9, 2019 and January 1, 2020.  The number of incoming and outgoing flights per day from each region will be analyzed in particular.

In [ ]:
# airport data from AZ
az_filter = df['ORIGIN_ST'].isin(['AZ']) | df['DEST_ST'].isin(['AZ'])
az_airport_df = filtered_df[az_filter]
az_airport_df.head()

In [ ]:
# number of flights that arrived at AZ in past year
az_airport_df_arrival = az_airport_df[az_airport_df["DEST_ST"] == 'AZ']
print(f"{az_airport_df_arrival.shape[0]} flights arrived at AZ in past year")

In [ ]:
# count number of flights per day that arrive at AZ
az_arrival_flights_per_day = az_airport_df_arrival.groupby("FL_DATE", observed=True).size()

**AZ Arrival Flights Per Day Statistics**

In [ ]:
# calculate statistics for flight frequencies
az_mean_arrival_flights = az_arrival_flights_per_day.mean()
az_median_arrival_flights = az_arrival_flights_per_day.median()
az_mode_arrival_flights = az_arrival_flights_per_day.mode().iloc[0]
az_std_arrival_flights = az_arrival_flights_per_day.std()
print("AZ Arrival Flights Per Day Statistics:")
print(f"mean: {az_mean_arrival_flights}")
print(f"median: {az_median_arrival_flights}")
print(f"mode: {az_mode_arrival_flights}")
print(f"std: {az_std_arrival_flights}")

In [ ]:
# number of flights that departed from AZ in past year
az_airport_df_departure = az_airport_df[az_airport_df["ORIGIN_ST"] == 'AZ']
print(f"{az_airport_df_departure.shape[0]} flights departed from AZ in past year")

In [ ]:
# count number of flights per day that depart from AZ
az_departure_flights_per_day = az_airport_df_departure.groupby("FL_DATE", observed=True).size()

**AZ Departure Flights Per Day Statistics**

In [ ]:
# calculate statistics for flight frequencies
az_mean_departure_flights = az_departure_flights_per_day.mean()
az_median_departure_flights = az_departure_flights_per_day.median()
az_mode_departure_flights = az_departure_flights_per_day.mode().iloc[0]
az_std_departure_flights = az_departure_flights_per_day.std()
print("AZ Departure Flights Per Day Statistics:")
print(f"mean: {az_mean_departure_flights}")
print(f"median: {az_median_departure_flights}")
print(f"mode: {az_mode_departure_flights}")
print(f"std: {az_std_departure_flights}")

In [ ]:
# airport data from NV
nv_filter = df['ORIGIN_ST'].isin(['NV']) | df['DEST_ST'].isin(['NV'])
nv_airport_df = filtered_df[nv_filter]
nv_airport_df.head()

In [ ]:
# number of flights that arrived at NV in past year
nv_airport_df_arrival = nv_airport_df[nv_airport_df["DEST_ST"] == 'NV']
print(f"{nv_airport_df_arrival.shape[0]} flights arrived at NV in past year")

In [ ]:
# count number of flights per day that arrive at NV
nv_arrival_flights_per_day = nv_airport_df_arrival.groupby("FL_DATE", observed=True).size()

**NV Arrival Flights Per Day Statistics:**

In [ ]:
# calculate statistics for flight frequencies
nv_mean_arrival_flights = nv_arrival_flights_per_day.mean()
nv_median_arrival_flights = nv_arrival_flights_per_day.median()
nv_mode_arrival_flights = nv_arrival_flights_per_day.mode().iloc[0]
nv_std_arrival_flights = nv_arrival_flights_per_day.std()
print("NV Arrival Flights Per Day Statistics:")
print(f"mean: {nv_mean_arrival_flights}")
print(f"median: {nv_median_arrival_flights}")
print(f"mode: {nv_mode_arrival_flights}")
print(f"std: {nv_std_arrival_flights}")

In [ ]:
# number of flights that departed from NV in past year
nv_airport_df_departure = nv_airport_df[nv_airport_df["ORIGIN_ST"] == 'NV']
print(f"{nv_airport_df_departure.shape[0]} flights departed from NV in past year")

In [ ]:
# count number of flights per day that depart from NV
nv_departure_flights_per_day = nv_airport_df_departure.groupby("FL_DATE", observed=True).size()

**NV Departure Flights Per Day Statistics**

In [ ]:
# calculate statistics for flight frequencies
nv_mean_departure_flights = nv_departure_flights_per_day.mean()
nv_median_departure_flights = nv_departure_flights_per_day.median()
nv_mode_departure_flights = nv_departure_flights_per_day.mode().iloc[0]
nv_std_departure_flights = nv_departure_flights_per_day.std()
print("NV Departure Flights Per Day Statistics:")
print(f"mean: {nv_mean_departure_flights}")
print(f"median: {nv_median_departure_flights}")
print(f"mode: {nv_mode_departure_flights}")
print(f"std: {nv_std_departure_flights}")

In [ ]:
# airport data from CA
ca_filter = df['ORIGIN_ST'].isin(['CA']) | df['DEST_ST'].isin(['CA'])
ca_airport_df = filtered_df[ca_filter]
ca_airport_df.head()

In [ ]:
# number of flights that arrived at CA in past year
ca_airport_df_arrival = ca_airport_df[ca_airport_df["DEST_ST"] == 'CA']
print(f"{ca_airport_df_arrival.shape[0]} flights arrived at CA in past year")

In [ ]:
# count number of flights per day that arrive at AZ
ca_arrival_flights_per_day = ca_airport_df_arrival.groupby("FL_DATE", observed=True).size()

**CA Arrival Flights Per Day Statistics:**

In [ ]:
# calculate statistics for flight frequencies
ca_mean_arrival_flights = ca_arrival_flights_per_day.mean()
ca_median_arrival_flights = ca_arrival_flights_per_day.median()
ca_mode_arrival_flights = ca_arrival_flights_per_day.mode().iloc[0]
ca_std_arrival_flights = ca_arrival_flights_per_day.std()
print("CA Arrival Flights Per Day Statistics:")
print(f"mean: {ca_mean_arrival_flights}")
print(f"median: {ca_median_arrival_flights}")
print(f"mode: {ca_mode_arrival_flights}")
print(f"std: {ca_std_arrival_flights}")

In [ ]:
# number of flights that departed from AZ in past year
ca_airport_df_departure = ca_airport_df[ca_airport_df["ORIGIN_ST"] == 'CA']
print(f"{ca_airport_df_departure.shape[0]} flights departed from CA in past year")

In [ ]:
# count number of flights per day that depart from CA
ca_departure_flights_per_day = ca_airport_df_departure.groupby("FL_DATE", observed=True).size()

**CA Departure Flights Per Day Statistics**

In [ ]:
# calculate statistics for flight frequencies
ca_mean_departure_flights = ca_departure_flights_per_day.mean()
ca_median_departure_flights = ca_departure_flights_per_day.median()
ca_mode_departure_flights = ca_departure_flights_per_day.mode().iloc[0]
ca_std_departure_flights = ca_departure_flights_per_day.std()
print("CA Departure Flights Per Day Statistics:")
print(f"mean: {ca_mean_departure_flights}")
print(f"median: {ca_median_departure_flights}")
print(f"mode: {ca_mode_departure_flights}")
print(f"std: {ca_std_departure_flights}")

**Visualize AZ, NV, and CA Flights Per Day**

In [ ]:
flights_per_day_arrival = {
    'State': ['AZ', 'NV', 'CA'],
    'Mean': [az_mean_arrival_flights, nv_mean_arrival_flights, ca_mean_arrival_flights],
    'Median': [az_median_arrival_flights, nv_median_arrival_flights, ca_median_arrival_flights],
    'Mode': [az_mode_arrival_flights, nv_mode_arrival_flights, ca_mode_arrival_flights],
    'Std Dev': [az_std_arrival_flights, nv_std_arrival_flights, ca_std_arrival_flights]
}

flights_per_day_departure = {
    'State': ['AZ', 'NV', 'CA'],
    'Mean': [az_mean_departure_flights, nv_mean_departure_flights, ca_mean_departure_flights],
    'Median': [az_median_departure_flights, nv_median_departure_flights, ca_median_departure_flights],
    'Mode': [az_mode_departure_flights, nv_mode_departure_flights, ca_mode_departure_flights],
    'Std Dev': [az_std_departure_flights, nv_std_departure_flights, ca_std_departure_flights]
}

flights_per_day_arrival_df = pd.DataFrame(flights_per_day_arrival)
flights_per_day_departure_df = pd.DataFrame(flights_per_day_departure)

**Flights Per Day Arrival**

In [ ]:
flights_per_day_arrival_df.index = ['AZ', 'NV', 'CA']
flights_per_day_arrival_df.plot(y=["Mean", "Median", "Mode", "Std Dev"], kind="bar")
plt.title("Daily Arrival Flights Per Region")
plt.ylabel("Number of Flights")
plt.xlabel("Region")
plt.show()

**Flights Per Day Departure**

In [ ]:
flights_per_day_departure_df.index = ['AZ', 'NV', 'CA']
flights_per_day_departure_df.plot(y=["Mean", "Median", "Mode", "Std Dev"], kind="bar")
plt.title("Daily Departure Flights Per Region")
plt.ylabel("Number of Flights")
plt.xlabel("Region")
plt.show()

**Analysis**

Based on the provided data, CA has the most air traffic as it has the greatest average arrival and departure flights per day.  CA has an average of 2183 arrival and 2188 departure flights daily.  AZ has an average of 552 arrival and 552 departure flights daily, while NV has an average of 498 arrival and 498 departure flights daily.

Based on the graph results, CA clearly dominates in terms of number of flights and has a greater standard deviation, indicating more daily variability than the other two regions.  AZ and NV have similar means and medians with low standard deviations, implying more consistent day to day air traffic.  Since the mean, median, and mode are similar for all regions in terms of arrival and departure flights per day, this means that there are no extreme daily flight flight values affecting the average.

In terms of total number of flights, AZ, NV, and CA had 201744, 182115, and 798690 arrival flights in the past year respectively.  AZ, NV, and CA had 201804, 181985, and 799137 departure flights in the past year respectively.  This further proves that CA has the greatest air traffic, followed by AZ and then NV based on measures of daily fights.  


2. For each region, the 3 states (AZ, NV, CA), analyze the MOST POPULAR outbound/destination airports. For example, if a flight originated in CA (at any of its airports), where do they often go? Comment on your findings and visualize the results for the top 5 destinations.

**Average Arrival Delay by Carrier**

In [ ]:
import matplotlib.pyplot as plt

states = ['AZ', 'CA', 'NV']

def top_destinations_by_state(df, top_n=5):
    results = {}
    for state in states:
        subset = df[df['ORIGIN_ST'] == state]
        top_dest = (
            subset['DEST']
            .value_counts()
            .head(top_n)
            .to_frame(name='FLIGHT_COUNT')
            .reset_index()
            .rename(columns={'index': 'DEST'})
        )
        results[state] = top_dest

        plt.figure()
        plt.bar(top_dest['DEST'], top_dest['FLIGHT_COUNT'])
        plt.title(f"Top {top_n} Destinations from {state}")
        plt.xlabel("Destination Airport")
        plt.ylabel("Number of Flights")
        plt.tight_layout()
        plt.show()
    return results

top_destinations_by_state(df, top_n=5)

**Analysis**

The most common outbound destinations show strong regional travel corridors. From Arizona, flights most frequently go to Denver and Los Angeles, while Nevada heavily favors flights to Los Angeles and San Francisco, reflecting strong California-bound demand. California sends large volumes to Phoennix, Las Vegas, and Seattle, showing intra-western connectivity. These results confirm that western US routes dominate traffic patterns from these states.

3. For each region, calculate the PROPORTION of flights from each airline/operator. Visualize the top 10 results. Explain the results.

Hint: Don't forget that proportion is an explicit requirement

**Top 10 Airline Proportions by State**

In [ ]:
def airline_proportions(df, top_n=10):
    proportions_by_state = {}
    for state in states:
        subset = df[df['ORIGIN_ST'] == state]
        counts = subset['CARRIER_CODE'].value_counts(normalize=True).head(top_n)
        proportions = counts.to_frame(name='PROPORTION').reset_index().rename(columns={'index': 'CARRIER_CODE'})
        proportions_by_state[state] = proportions

        # Plot
        plt.figure()
        plt.bar(proportions['CARRIER_CODE'], proportions['PROPORTION'])
        plt.title(f"Top {top_n} Airline Proportions from {state}")
        plt.xlabel("Airline Code")
        plt.ylabel("Proportion of Flights")
        plt.ylim(0, proportions['PROPORTION'].max() + 0.05)
        plt.tight_layout()
        plt.show()

    return proportions_by_state

airline_proportions(df, top_n=10)

The airline proportions by region reveal that Southwest Airlines (WN) dominates in all three states, especially in Nevada where it operates over 42% of all outbound flights. California has the most diversified market, which multiple carriers (WN, OO, UA, AA) share the volume more evenly, while regional airlines like SkyWest (OO) and Mesa (YV) also have a significant presence in AZ and CA. These patterns suggest differences in market share and operational strategy by region.

## Question 4

1. Explore the carriers within the dataset and demonstrate at least TWO (2) patterns that you identified.

**Averge Arrival Delay by Carrier**

In [ ]:
def avg_arrival_delay(df):
    delay_summary = df.groupby('CARRIER_CODE', observed=True)['ARR_DELAY'].mean().sort_values()
    delay_summary.plot(kind='bar', figsize=(10, 5))
    plt.title("Average Arrival Delay by Carrier")
    plt.xlabel("Carrier Code")
    plt.ylabel("Average Arrival Delay (minutes)")
    plt.axhline(0, color='black', linestyle='--', linewidth=1)
    plt.tight_layout()
    plt.show()
    return delay_summary

avg_arrival_delay(df)

This graph shows the average arrival and departure delays for each airline, highlighting which carriers are most prone to delays.

**Missing Arrival or Elapsed Time by Carrer**

In [ ]:
def missing_flight_data(df):
    missing_mask = df['ARR_TIME'].isnull() | df['ELAPSED_TIME'].isnull()
    missing_counts = df[missing_mask].groupby('CARRIER_CODE', observed=True).size().sort_values(ascending=False)
    missing_counts.plot(kind='bar', figsize=(10, 5))
    plt.title("Missing Arrival or Elapsed Time by Carrier")
    plt.xlabel("Carrier Code")
    plt.ylabel("Missing Flight Records")
    plt.tight_layout()
    plt.show()
    return missing_counts

missing_flight_data(original_df)

This graph shows the number of flights with missing arrival or elapsed time per airline, which can indicate data quality or operational issues.

**Departure Time Distribution by Carrier**

In [ ]:
flight_df_dep = df.copy()

def categorize_dep_hour(dep_time):
    if pd.isna(dep_time):
        return 'Unknown'
    hour = int(dep_time) // 100
    if hour < 6:
        return 'Early Morning (0-6AM)'
    elif hour < 12:
        return 'Morning (6AM - 12PM)'
    elif hour < 18:
        return 'Afternoon (12 - 6PM)'
    else:
        return 'Evening (6PM - Midnight)'

flight_df_dep['DEP_PERIOD'] = flight_df_dep['DEP_TIME'].apply(categorize_dep_hour)

def departure_patterns(df, carriers=None):
    if carriers:
        df = df[df['CARRIER_CODE'].isin(carriers)]
    grouped = df.groupby(['CARRIER_CODE', 'DEP_PERIOD'], observed=True).size().unstack(fill_value=0)
    proportions = grouped.div(grouped.sum(axis=1), axis=0)
    proportions.plot(kind='bar', stacked=True, figsize=(12, 6))
    plt.title("Departure Time Distribution by Carrier")
    plt.ylabel("Proportion of Flights")
    plt.xlabel("Carrier")
    plt.legend(title="Departure Period")
    plt.tight_layout()
    plt.show()
    return proportions

departure_patterns(flight_df_dep)

This stacked bar chart shows the proportion of flights for each airline across different departure time periods, helping identify scheduling tendencies.

2. Analyze the MINIMUM, MAXIMUM, and AVERAGE ARRIVAL AND DEPARTURE DELAYS for each Airline/Carrier. Visualize the top 10 results. Explain the patterns in the delays and demonstrate: 1) which carriers are more prone to flight delays and 2) are the delays occurring more frequently in certain regions**?

**TIP: it is important to analyze the airlines across multiple airports and/or regions in order to conclude that they have a pattern of being late.

**Top 10 carriers by Average Arrival and Departure Delays**

In [ ]:
def avg_delay_stats_by_carrier(df, top_n=10):
    delay_df = df.dropna(subset=['ARR_DELAY', 'DEP_DELAY'])

    stats = delay_df.groupby('CARRIER_CODE', observed=True)[['ARR_DELAY', 'DEP_DELAY']].agg(['min', 'max', 'mean'])
    stats.columns = ['ARR_MIN', 'ARR_MAX', 'ARR_MEAN', 'DEP_MIN', 'DEP_MAX', 'DEP_MEAN']
    stats = stats.sort_values(by='ARR_MEAN', ascending=False)

    top_stats = stats.head(top_n)

    top_stats[['ARR_MEAN', 'DEP_MEAN']].plot(kind='bar', figsize=(12, 6))
    plt.title(f"Top {top_n} Carriers by Average Arrival and Departure Delays")
    plt.ylabel("Average Delay (minutes)")
    plt.xlabel("Carrier Code")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

    return top_stats

avg_delay_stats = avg_delay_stats_by_carrier(df, top_n=10)
print(avg_delay_stats)

**Top 10 States by Average Arrival and Departure Delays**

In [ ]:
def regional_delay_patterns(df, top_n=10):
    regional_df = df.dropna(subset=['ARR_DELAY', 'DEP_DELAY', 'ORIGIN_ST'])

    state_delays = regional_df.groupby('ORIGIN_ST', observed=True)[['ARR_DELAY', 'DEP_DELAY']].mean()
    state_delays.columns = ['AVG_ARR_DELAY', 'AVG_DEP_DELAY']

    top_states = state_delays.sort_values(by='AVG_ARR_DELAY', ascending=False).head(top_n)

    top_states.plot(kind='bar', figsize=(12, 6))
    plt.title(f"Top {top_n} States by Average Arrival and Departure Delays")
    plt.xlabel("Origin State")
    plt.ylabel("Average Delay (minutes)")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

    return top_states

regional_delay = regional_delay_patterns(df, top_n=10)
print(regional_delay)

**Average Arrival Delay by Carrier and State**

In [ ]:
import seaborn as sns

def arr_delay_patterns_by_airline_and_region(df, top_carriers=None, top_states=None):
    delay_data = df.dropna(subset=['ARR_DELAY', 'CARRIER_CODE', 'ORIGIN_ST'])

    grouped = delay_data.groupby(['CARRIER_CODE', 'ORIGIN_ST'], observed=True)['ARR_DELAY'].mean().reset_index()

    if top_carriers:
        grouped = grouped[grouped['CARRIER_CODE'].isin(top_carriers)]
    if top_states:
        grouped = grouped[grouped['ORIGIN_ST'].isin(top_states)]

    pivot = grouped.pivot(index='CARRIER_CODE', columns='ORIGIN_ST', values='ARR_DELAY').fillna(0)

    pivot = pivot.astype(float)

    plt.figure(figsize=(14, 6))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="coolwarm", linewidths=0.5)
    plt.title("Average Arrival Delay by Carrier and State")
    plt.xlabel("Origin State")
    plt.ylabel("Carrier Code")
    plt.tight_layout()
    plt.show()

    return pivot

# extract top 10 arrival carriers
top_arr_carriers = avg_delay_stats.sort_values(by='ARR_MEAN', ascending=False).head(10).index.tolist()

# extract top 10 arrival states
top_arr_states = regional_delay.sort_values(by='AVG_ARR_DELAY', ascending=False).head(10).index.tolist()

arr_delay_patterns_by_airline_and_region(df, top_arr_carriers, top_arr_states)


**Average Depature Delay by Carrier and State**

In [ ]:
def dep_delay_patterns_by_airline_and_region(df, top_carriers=None, top_states=None):
    delay_data = df.dropna(subset=['DEP_DELAY', 'CARRIER_CODE', 'ORIGIN_ST'])

    grouped = delay_data.groupby(['CARRIER_CODE', 'ORIGIN_ST'])['DEP_DELAY'].mean().reset_index()

    if top_carriers:
        grouped = grouped[grouped['CARRIER_CODE'].isin(top_carriers)]
    if top_states:
        grouped = grouped[grouped['ORIGIN_ST'].isin(top_states)]

    pivot = grouped.pivot(index='CARRIER_CODE', columns='ORIGIN_ST', values='DEP_DELAY').fillna(0)

    pivot = pivot.astype(float)

    plt.figure(figsize=(14, 6))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="coolwarm", linewidths=0.5)
    plt.title("Average Departure Delay by Carrier and State")
    plt.xlabel("Origin State")
    plt.ylabel("Carrier Code")
    plt.tight_layout()
    plt.show()

    return pivot

# extract top 10 departure carriers
top_dep_carriers = avg_delay_stats.sort_values(by='DEP_MEAN', ascending=False).head(10).index.tolist()

# extract top 10 departure states
top_dep_states = regional_delay.sort_values(by='AVG_DEP_DELAY', ascending=False).head(10).index.tolist()

dep_delay_patterns_by_airline_and_region(df, top_dep_carriers, top_dep_states)

In [ ]:
# filter flights operated by AA that originated in ND
# exclude any records with missing arrival or departure delay data
aa_nd_flights = df[
    (df['CARRIER_CODE'] == 'AA') &
    (df['ORIGIN_ST'] == 'ND') &
    (~df['ARR_DELAY'].isna()) &
    (~df['DEP_DELAY'].isna())
]

# calculate how many such flights exist and summarize their delay statistics.
aa_nd_count = len(aa_nd_flights)
aa_nd_summary = aa_nd_flights[['ARR_DELAY', 'DEP_DELAY']].describe()

aa_nd_count, aa_nd_summary

1) Regional and low-cost carriers such as Republic Airways (YX), Frontier Airlines (F9), and Mesa Airlines (YV) are more prone to flight delays, with the highest average arrival and departure delays across the dataset. These airlines typically operate on tighter schedules with limited fleet flexibility, making them more vulnerable to disruption. In contrast, larger carriers like Alaska (AS) and Hawaiian (HA) demonstrated more reliable performance, with consistently lower average delays.

2) Yes, delays are more frequent in specific regions. States like Wyoming (WY), Montana (MT), and New Jersey (NJ) consistently showed high average delays, driven by factors such as weather conditions or airspace congestion. One notable case was American Airlines in North Dakota (ND), which showed extremely high average delays. However, this was based on only 13 flights and skewed by outliers, so it shouldn't be interpreted as a consistent pattern.

## Question 5

In [ ]:
performance = df.groupby("CARRIER_CODE", observed=False).apply(lambda x: pd.Series({
    "OnTime_Arrival_15min_%": (x["ARR_DELAY"] <= 15).mean() * 100,
    "OnTime_Departure_%": (x["DEP_DELAY"] <= 0).mean() * 100,
    "Total_Flights": x.shape[0]
})).reset_index()

metrics = ["OnTime_Arrival_15min_%", "OnTime_Departure_%"]
performance[metrics] = performance[metrics].apply(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
print("\nAfter normalization:")
print(performance.round(2).to_string(index=False))

In [ ]:
performance["Performance_Score"] = (
    0.6 * performance["OnTime_Arrival_15min_%"] +
    0.4 * performance["OnTime_Departure_%"]
)
print("\nAfter performance score calculation:")
print(performance.round(2).to_string(index=False))

In [ ]:
top_airlines = performance[performance["Total_Flights"] >= 100]\
    .nlargest(10, "Performance_Score")\
    [["CARRIER_CODE", "Performance_Score", "OnTime_Arrival_15min_%", "OnTime_Departure_%"]]

print("Top 10 Airlines by Operational Punctuality:")
print(top_airlines.round(2).to_string(index=False))

I evaluated which airlines had the best performance by creating a composite score based on common aviation KPIs. The two indicators I used here were punctuality in terms of on time arrivals within 15 minutes and ontime departures; I created an aggregate of these metrics, weighting arrivals slightly more important in terms of performance. The calculations suggested that the airline with the carrier codes YX and 00 have the best performance, using the two measures of punctuality.

In [ ]:
top_10_codes = top_airlines['CARRIER_CODE'].unique().tolist()
df_top10 = df[df['CARRIER_CODE'].isin(top_10_codes)].copy()

In [ ]:
df_top10['Month'] = pd.to_datetime(df_top10['FL_DATE']).dt.strftime('%Y-%m')
flight_hours_monthly = (df_top10.groupby(['CARRIER_CODE', 'Month'], observed=False)
                         .agg(Total_Flight_Minutes=('ELAPSED_TIME', 'sum'))
                         .reset_index())
flight_hours_monthly['Total_Flight_Hours'] = flight_hours_monthly['Total_Flight_Minutes'] / 60

In [ ]:
pivot_data = (flight_hours_monthly.pivot(index='Month', 
                                       columns='CARRIER_CODE', 
                                       values='Total_Flight_Hours')
              [top_10_codes])

In [ ]:
plt.figure(figsize=(14, 8))
ax = pivot_data.plot(kind='bar', width=0.8)
plt.title('Monthly Flight Hours for Top 10 Airlines by Performance', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Flight Hours', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Airline', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

This visualization shows the monthly flight hours for the top 10 airlines, as determined by the performance scores. Out of the top 10, UA had by far the most flight hours by month. This data could be further factored in to create a new performance score, which reflects flight volume/hours flown, as that is an indicator of a more active/prominent airline.

## Question 6

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = '2019_ONTIME_REPORTING_FSW-1.csv'
df = pd.read_csv(file_path)

selected_tail_nums = ['N972NN', 'N352PS', 'N305NX']
filtered_df = df[df['TAIL_NUM'].isin(selected_tail_nums)].copy()

for col in ['DEP_DELAY', 'ARR_DELAY', 'DISTANCE']:
    filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce')

# --- 1. Summary statistics ---
grouped = filtered_df.groupby('TAIL_NUM')
summary = grouped.agg({
    'DEP_DELAY': ['mean', 'max', 'min'],
    'ARR_DELAY': ['mean', 'max', 'min'],
    'DISTANCE': ['mean', 'max', 'min']
}).round(2)

print("Summary Statistics for Selected Aircraft:")
print(summary)

# --- 2. Bar plot for average delays ---
summary_delay = summary[['DEP_DELAY', 'ARR_DELAY']].xs('mean', level=1, axis=1)
summary_delay.plot(kind='bar', figsize=(10,6))
plt.title('Average Departure and Arrival Delays for Selected Aircraft')
plt.ylabel('Delay (minutes)')
plt.xlabel('Aircraft TAIL_NUM')
plt.xticks(rotation=0)
plt.legend(['Departure Delay', 'Arrival Delay'])
plt.tight_layout()
plt.show()

# --- 3. Bar plot for average distance ---
summary_distance = summary['DISTANCE']['mean']
summary_distance.plot(kind='bar', figsize=(8,5), color='green')
plt.title('Average Distance Traveled for Selected Aircraft')
plt.ylabel('Distance (miles)')
plt.xlabel('Aircraft TAIL_NUM')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# --- 4. Top routes and delay breakdowns ---
for tail_num in selected_tail_nums:
    print(f"\nTop routes for {tail_num}:")
    aircraft_data = filtered_df[filtered_df['TAIL_NUM'] == tail_num]
    top_routes = aircraft_data.groupby(['ORIGIN', 'DEST']).size().reset_index(name='count').sort_values(by='count', ascending=False)
    print(top_routes.head(5))
    
    # Create a new 'Route' column combining ORIGIN and DEST
    top_routes['Route'] = top_routes['ORIGIN'] + ' - ' + top_routes['DEST']
    
    # Bar chart of top routes
    top_routes.head(5).plot(kind='bar', x='Route', y='count', figsize=(10,5), title=f'Top Routes for {tail_num}')
    plt.ylabel('Number of Flights')
    plt.tight_layout()
    plt.show()
    
    # Delay breakdown by top routes
    delay_by_route = aircraft_data.groupby(['ORIGIN', 'DEST']).agg({
        'DEP_DELAY':'mean',
        'ARR_DELAY':'mean'
    }).reset_index().sort_values(by='DEP_DELAY', ascending=False)
    
    # Create a new 'Route' column combining ORIGIN and DEST
    delay_by_route['Route'] = delay_by_route['ORIGIN'] + ' - ' + delay_by_route['DEST']
    
    print(f"\nMean delays by route for {tail_num}:")
    print(delay_by_route.head(5))
    
    # Bar chart for average delays by route
    delay_by_route.head(5).plot(kind='bar', x='Route', y=['DEP_DELAY','ARR_DELAY'], figsize=(10,5), title=f'Mean Delays by Route for {tail_num}')
    plt.ylabel('Delay (minutes)')
    plt.tight_layout()
    plt.show()



## Question 7

In [ ]:
from IPython.display import Markdown

summary_text = """
#  **Question 7 – Overall Analysis Summary**

After conducting a comprehensive analysis of the 2019 On-Time Performance dataset for flights originating from Arizona (AZ), Nevada (NV), and California (CA), we have identified several key insights:

---

###  **Key Finding 1: California Has the Most Air Traffic (Question 3)**  
Our analysis revealed that **California (CA)** had the highest air traffic among AZ, NV, and CA, determined by the frequency of flights. This reflects both the state's population size and its role as a major hub for domestic travel.

---

###  **Key Finding 2: American Airlines Operates Most Selected Aircraft with Moderate to High Delays (Questions 4 & 6)**  
In our carrier analysis (Question 4), **American Airlines (AA)** emerged as a key operator, with varying levels of delays. The detailed aircraft analysis (Question 6) of three specific aircraft (**N972NN, N352PS, N305NX**) showed:  
- High average delays (up to **17.61 mins**) and variable maximum delays (up to 556 mins).  
- Delays were notably higher on busy routes such as **DFW-SNA** and **SNA-ORD**, pointing to congestion and operational challenges.

---

###  **Key Finding 3: Distance and Delays Show a Weak Correlation (Question 6)**  
The **distance** field in the dataset represents the miles flown per flight segment (between ORIGIN and DEST). For each aircraft, the average distance traveled per flight was approximately **1,426 to 1,487 miles**.  
- Even with similar average distances, delays varied significantly across flights.  
- This highlights that factors such as airport congestion, weather, and operational issues likely have a larger impact on delays than the distance flown.

---

###  **Clarification on Distance Calculation**  
Each row in the dataset corresponds to an individual flight, and the **distance** field represents the distance covered on that specific flight. The average distance we calculated per aircraft reflects the **mean distance of all its flights**, not the total distance flown by the aircraft.  
- To extend the analysis, we could calculate the **total distance covered by each aircraft** by summing the distance values for all its flights.

---

### 📊 **Visualizations and Analysis**  
Throughout the practicum, we created visualizations with clear titles and labeled axes:  
- **Bar plots** for average delays and distances by aircraft (Questions 4 & 6)  
- **Boxplots** showing delay variability by aircraft (Question 6)  
- **Proportional analysis** of airlines in each region (Question 3)

---

### 👥 **Teammates**  
Jasmine Chen, Simran Sharma, Gauri Rejesh, Nathanael Dougherty

"""

# Display the markdown summary
display(Markdown(summary_text))

